In [ ]:
""" select the best module for each signature
"""

In [2]:
import sys
sys.path.append('../')
import os
import pandas as pd
from importlib import reload  # Python 3.4+ only.
import statsmodels.stats.multitest as multitest
import scipy.stats as ss
import numpy as np
import networkx as nx

home_dir="/home/kimy3/Projects/NetPhix/NetPhix/"
result_dir = home_dir+"results/"

In [9]:
# read all modules and compute FDR 
pvs_all= []
rows = []
sig_list = ['2C', '2D', '3C', '3D','8C', '13C', '13D']
for sig in sig_list:
    print(sig)
    for k in range(1, 8):       
        sol_name = result_dir+"netphix_results"+sig+".txt"
        if os.path.isfile(sol_name) is False:
            print("not found")
            continue
        solution_dics, cost, p  = netphix.read_solutionfile(sol_name)
        if len(solution_dics) < 1:
            continue
        pvs_all.append(p)
        selected_muts = solution_dics[0]["selected_muts"]
        time = solution_dics[0]["time"]
        rows.append((sig,  k, cost, ",".join(selected_muts), time, p))
            
columns = ["sig", "k", "TotCost",  "selected", "time", "pv"]
modules_df = pd.DataFrame(rows, columns = columns) 
fdr_results = multitest.multipletests(pvs_all, alpha=0.1, method="fdr_bh")
modules_df["adj-p(BH)"] = fdr_results[1]
modules_df["FDR <= 0.1"] = fdr_results[0]

modules_df['TotCost'] = modules_df['TotCost'].map(lambda x: '%2.1f' % x)
modules_df['pv'] = modules_df['pv'].map(lambda x: '%.4f' % x)
modules_df['adj-p(BH)'] = modules_df["adj-p(BH)"].map(lambda x: '%.4f' % x)


2C
2D
3C
3D
8C
13C
13D


In [29]:
# select best modules for each signature
def select_best_module(exp, itr_th, pv_th):
    """
    exp: modules with different parameteres for a sig
    itr_th: improvement threshold for each iteration
    pv_th: pvalue threshold 
    """
    exp.sort_values("k")
    costs = exp["TotCost"].astype(float).values
    pvs = exp["pv"].astype(float).values
    cost_dif = [costs[j]/costs[j-1] -1 for j in range(1, len(costs))]
    for j in range(5):
        k1 = j+1
        if (j == 4) or (cost_dif[j] <= itr_th) or (pvs[j+1] > pvs[j]):  # improvement is not significant or pv getting worse
            if (pvs[j] < pv_th):
                print("sig at "+str(k1))
                return exp.iloc[j,:]
            else:
                print("stop at "+str(k1)+" but not sig")
                return None
            break

cost_dic, pvs_dic = {}, {}
rows = []
itr_th, pv_th = 0.05,0.05
# DF for selected modules
selected_df = pd.DataFrame(columns=columns+["adj-p(BH)"])

for sig in sig_list:
    print(sig)
    exp = modules_df[(modules_df["sig"] == sig)]
    selected = select_best_module(exp, itr_th, pv_th)
    if selected is not None:
        selected_df = selected_df.append(selected,ignore_index=True)

2C
sig at 3
2D
sig at 3
3C
sig at 3
3D
sig at 2
8C
sig at 2
13C
sig at 3
13D
sig at 3
